In [1]:
import os
import librosa
import numpy as np
import resampy

def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

def load_data(data_dir):
    features = []
    labels = []
    
    for folder in ['lowSpeed', 'HighSpeed', 'Abnormal']:
        folder_path = os.path.join(data_dir, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            data = extract_features(file_path)
            features.append(data)
            labels.append(folder)
    
    return np.array(features), np.array(labels)

data_dir = './dataset/'
X, y = load_data(data_dir)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Abnormal       1.00      1.00      1.00         2
   HighSpeed       1.00      1.00      1.00         2
    lowSpeed       1.00      1.00      1.00         2

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



In [3]:
import sounddevice as sd
from matplotlib import pyplot as plt

def predict_live_sound(model, duration=5, sample_rate=22050):
    print("Recording...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    # Plot the waveform
    sd.wait()
    
    print("Recording complete.")
    # plt.figure(figsize=(10, 4))
    # plt.plot(recording)
    # plt.title("Live Audio Waveform")
    # plt.xlabel("Time")
    # plt.ylabel("Amplitude")
    # plt.show(block=False)
    # plt.pause(0.1)  # Adjust this if the plot does not update properly
    # plt.close()
    # Extract features from the recorded audio
    mfccs = librosa.feature.mfcc(y=recording.flatten(), sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    mfccs_scaled = mfccs_scaled.reshape(1, -1)
    
    # Predict the class
    prediction = model.predict(mfccs_scaled)
    print("Predicted class:", prediction[0])

# Example usage

try:
    while True:
        predict_live_sound(clf)
except KeyboardInterrupt:
    print("Stopped by user.")


Recording...
Recording complete.
Predicted class: lowSpeed
Recording...
Recording complete.
Predicted class: HighSpeed
Recording...
Recording complete.
Predicted class: lowSpeed
Recording...
Recording complete.
Predicted class: lowSpeed
Recording...
Stopped by user.
